In [1]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd 
import numpy as np
import seaborn as sns
from re import sub
import json
import requests
import time
from bs4 import BeautifulSoup as bs

sns.set_style("darkgrid")

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm

In [3]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

# Read CSVs

In [4]:
mov_df1 = pd.read_csv("data/en_csv.csv")
mov_df2 = pd.read_csv("data/five_year.csv")
dsc_df = pd.read_csv("data/discover.csv", engine='python')

scrape = pd.read_csv("data/scrape.csv")


## Concatenate DataFrames

In [5]:
mov12 = pd.concat([mov_df1, mov_df2, dsc_df], sort=False)

In [6]:
mov12.shape

(110059, 31)

In [7]:
scrape.drop(['Unnamed: 0'], axis = 1, inplace=True)
scrape.drop_duplicates(inplace=True)

In [8]:
dfm = mov12.merge(scrape, how='left', on="imdb_id")

In [9]:
dfm.loc[dfm.budget_x == 0,'budget_x'] = dfm.loc[dfm.budget_x == 0,'budget_y']
dfm.loc[dfm.revenue_x == 0,'revenue_x'] = dfm.loc[dfm.revenue_x == 0,'revenue_y']
dfm.columns

Index(['Unnamed: 0', 'adult', 'backdrop_path', 'belongs_to_collection',
       'budget_x', 'genres', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue_x', 'runtime', 'spoken_languages', 'status', 'tagline',
       'title', 'video', 'vote_average', 'vote_count', 'success',
       'status_code', 'status_message', 'Unnamed: 0.1', 'genre_ids',
       'budget_y', 'revenue_y'],
      dtype='object')

In [10]:
cols = ['Unnamed: 0', 'adult', 'backdrop_path', 'belongs_to_collection',
       'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline',
       'title', 'video', 'vote_average', 'vote_count', 'success',
       'status_code', 'status_message', 'Unnamed: 0.1', 'genre_ids',
       'budget_y', 'revenue_y']
dfm.columns = cols

In [11]:
dfm.dropna(subset=['genres', 'budget', 'revenue', 'vote_average'], inplace=True)
dfm.shape

(6956, 33)

In [12]:
dfm.describe()

,budget,id,revenue,runtime,vote_average,vote_count,status_code,Unnamed: 0.1,budget_y,revenue_y
count,6.956000e+03,6956.000000,6.956000e+03,6923.000000,6956.000000,6956.000000,0.0,2309.000000,1.328000e+03,1.328000e+03
mean,6.151782e+07,225750.039965,9.601579e+07,106.776686,6.297513,1698.287809,NaN,2350.071460,1.640326e+08,6.114350e+06
std,7.882176e+08,221475.049980,1.914043e+08,25.342162,1.419948,2778.436028,NaN,2544.761509,1.798441e+09,2.794558e+07
min,0.000000e+00,5.000000,1.000000e+00,0.000000,0.000000,0.000000,NaN,0.000000,0.000000e+00,3.700000e+01
25%,4.772375e+06,10423.750000,2.827248e+06,94.000000,5.900000,138.000000,NaN,280.000000,8.236450e+05,2.781225e+04
50%,1.700000e+07,157353.500000,2.724381e+07,105.000000,6.400000,662.000000,NaN,1293.000000,3.000000e+06,2.385235e+05
75%,4.200000e+07,428744.000000,1.004898e+08,120.000000,7.100000,1986.750000,NaN,3743.000000,1.200000e+07,2.230020e+06
max,3.500000e+10,761122.000000,2.797801e+09,254.000000,10.000000,27545.000000,NaN,9972.000000,3.500000e+10,4.511834e+08


In [13]:
dfm.head()

,Unnamed: 0,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,video,vote_average,vote_count,success,status_code,status_message,Unnamed: 0.1,genre_ids,budget_y,revenue_y
9,9,False,/2nFzxaAK7JIsk6l7qZ8rFBsa3yW.jpg,"{'id': 619537, 'name': 'Train to Busan Collect...",17000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",https://www.wellgousa.com/films/train-to-busan...,581392.0,tt8850222,ko,...,False,7.0,732.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,False,/zzWGRw277MNoCs3zhyG3YmYQsXv.jpg,NaN,200000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://movies.disney.com/mulan-2020d,337401.0,tt4566758,en,...,False,7.2,2809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,13,False,/wu1uilmhM4TdluKi2ytfz8gidHf.jpg,"{'id': 275402, 'name': 'SpongeBob Collection',...",60000000.0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 16, 'na...",https://www.spongebobmovie.com/,400160.0,tt4823776,en,...,False,8.3,1055.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,32,False,/zogWnCSztU8xvabaepQnAwsOtOt.jpg,NaN,30000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,531499.0,tt8461224,en,...,False,6.0,171.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,37,False,/sizHX5VbwlBihaathTQHVGk1jdi.jpg,"{'id': 604246, 'name': 'Attraction Collection'...",8819200.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,514207.0,tt8060328,ru,...,False,7.0,415.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
dfm = dfm.loc[dfm['budget'] > 101, :]
dfm = dfm.loc[dfm['revenue'] > 101, :]
dfm = dfm.loc[dfm['vote_average'] > .01, :]

In [15]:
dfm.shape

(6683, 33)

In [16]:
dfm['Unnamed: 0'].value_counts()

104     6
49      6
13      6
45      5
56      5
       ..
4752    1
4753    1
4758    1
375     1
3431    1
Name: Unnamed: 0, Length: 5381, dtype: int64

In [17]:
cols = ['imdb_id', 'title', 'genres', 'popularity', 'release_date', 'budget', 'revenue', 'vote_average' ]
cols_to_drop = [x for x in mov_df2.columns if x not in cols]

In [18]:
df = dfm.copy()
df.drop(columns = cols_to_drop, inplace=True)
df = df.reindex(columns=cols)
df

,imdb_id,title,genres,popularity,release_date,budget,revenue,vote_average
9,tt8850222,Peninsula,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",800.753,2020-07-15,17000000.0,35878266.0,7.0
10,tt4566758,Mulan,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",752.021,2020-09-04,200000000.0,57000000.0,7.2
13,tt4823776,The SpongeBob Movie: Sponge on the Run,"[{'id': 14, 'name': 'Fantasy'}, {'id': 16, 'na...",1114.88,2020-08-14,60000000.0,4700000.0,8.3
32,tt8461224,The Tax Collector,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",492.096,2020-08-07,30000000.0,942666.0,6.0
37,tt8060328,Invasion,"[{'id': 878, 'name': 'Science Fiction'}]",385.953,2020-01-01,8819200.0,32104054.0,7.0
...,...,...,...,...,...,...,...,...
57528,tt9239802,Qarakoz,"[{'id': 27, 'name': 'Horror'}]",1.04,2020-03-26,10000000.0,3885.0,4.0
57911,NaN,ALL COPS AIN'T BAD,"[{'id': 35, 'name': 'Comedy'}]",1.23,2020-01-01,100000.0,350000.0,10.0
59573,tt11985684,Rómulo y Julita,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",0.739,2020-02-27,200000.0,40534.0,6.0
59859,tt11579832,The Unethicals,"[{'id': 35, 'name': 'Comedy'}, {'id': 28, 'nam...",0.944,2020-01-06,129.0,129.0,10.0


In [19]:
df['ROI'] = df['revenue'] / df['budget']

In [20]:
df

,imdb_id,title,genres,popularity,release_date,budget,revenue,vote_average,ROI
9,tt8850222,Peninsula,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",800.753,2020-07-15,17000000.0,35878266.0,7.0,2.110486
10,tt4566758,Mulan,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",752.021,2020-09-04,200000000.0,57000000.0,7.2,0.285000
13,tt4823776,The SpongeBob Movie: Sponge on the Run,"[{'id': 14, 'name': 'Fantasy'}, {'id': 16, 'na...",1114.88,2020-08-14,60000000.0,4700000.0,8.3,0.078333
32,tt8461224,The Tax Collector,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",492.096,2020-08-07,30000000.0,942666.0,6.0,0.031422
37,tt8060328,Invasion,"[{'id': 878, 'name': 'Science Fiction'}]",385.953,2020-01-01,8819200.0,32104054.0,7.0,3.640246
...,...,...,...,...,...,...,...,...,...
57528,tt9239802,Qarakoz,"[{'id': 27, 'name': 'Horror'}]",1.04,2020-03-26,10000000.0,3885.0,4.0,0.000389
57911,NaN,ALL COPS AIN'T BAD,"[{'id': 35, 'name': 'Comedy'}]",1.23,2020-01-01,100000.0,350000.0,10.0,3.500000
59573,tt11985684,Rómulo y Julita,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",0.739,2020-02-27,200000.0,40534.0,6.0,0.202670
59859,tt11579832,The Unethicals,"[{'id': 35, 'name': 'Comedy'}, {'id': 28, 'nam...",0.944,2020-01-06,129.0,129.0,10.0,1.000000


In [21]:
df.to_csv("data/cleaned.csv", index= False)